**This problem set is due Wednesday, September 14, 2021 at 11:59 pm. Please plan ahead and submit your work on time.**

Import the modules needed for this exercise (make sure you execute the cell below by clicking on it and pressing Shift-Enter)

**Do not import any other modules**

In [22]:
%load_ext autoreload
%autoreload 2
from utils import test_ok, check_expanded_states, check_expanded_nodes
from search_classes import SearchNode, Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem Set 1: Solving the 8 Puzzle Problem with Search


1. [State representation in the 8 Puzzle Problem](#state_representation)
    1. [Expanding the puzzle state (25 points)](#state_expansion)
2. [Simple Search](#simple_search)
    1. [Completing the `PuzzleProblem` class (10 points)](#puzzle_problem_class)
    2. [Implementing Breath First Search (45 points)](#bfs_implementation)
    3. [Breadth First Search vs Depth First Search (20 points)](#bfs_vs_dfs)


In this Problem Set you will implement Breath First Search and use it to solve the [8 Puzzle Problem](https://en.wikipedia.org/wiki/15_puzzle).

The puzzle consists on a 3x3 grid with 8 numbered tiles and a missing tile. The objective consists on sliding the tiles around until all the numbered tiles are ordered and the missing tile stays at the lower right cell of the grid.

<img src="puzzle8.png"/>




## <a name="state_representation"></a>State representation in the 8 Puzzle Problem

To make things simple, we are giving you a possible state representation for the 8-puzzle problem.

We'll represent a given state of the puzzle by a tuple of three internal tuples. Each internal tuple represents a row of the puzzle. The missing tile is represented by $0$.

For example, the puzzle state below:

<img src="example_state.png"/>

is represented by `((1, 2, 3), (8, 0, 4), (7, 6, 5))`.

Below, we are giving you some code to print a puzzle state:

In [23]:
# (this cell will be overwritten)
def print_state(state):
    print("+"+ "-"*5+"+")
    for l in state:
        print("|"+ " ".join([str(el) if el!=0 else " " for el in l]) +"|")
    print("+"+ "-"*5+"+")

example_state = ((1, 2, 3), (8, 0, 4), (7, 6, 5))

print("%s state represents puzzle state: " % (example_state,))
print_state(example_state)

((1, 2, 3), (8, 0, 4), (7, 6, 5)) state represents puzzle state: 
+-----+
|1 2 3|
|8   4|
|7 6 5|
+-----+


### <a name="state_expansion"></a>Expanding a puzzle state (25 points)

In order to find a solution to the state, we need to define the states we can reach from a given state.

This corresponds to the possible moves of the missing tile (at most up, down, left and right).

Implement the function `expand_state(state)` that returns a `list` of the states that can be reached from the given `state`.

For example, for state `((0, 1, 3), (4, 2, 5), (7, 8, 6))`, the function `expand_state` should return the following list (two moves are feasible):

```
[((4, 1, 3), (0, 2, 5), (7, 8, 6)), ((1, 0, 3), (4, 2, 5), (7, 8, 6))]
```

The neighbour states of state:

```
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+
```

are:

```
+-----+
|4 1 3|
|  2 5|
|7 8 6|
+-----+
***
+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+
```

<div class="alert alert-info">
Implement the function `expand_state(state)` below.
</div>


In [7]:
def expand_state(state):
    state = list(state)
    expand_state = []
    # find where the zero is
    for i,row in enumerate(state): # looks at column element within a row
        for j,el in enumerate(row): # looks at row element within the first row
            if el == 0:
                ii = i
                jj = j
    #cycle through each action
    #right = [[0,1]] #action, moves the zero in state to the adjacent indice
    #left = [[0,-1]]
    #up = [[1,0]]
    #down = [[-1,0]]
    actions = [[0,1],[0,-1],[1,0],[-1,0]]
    for i,row in enumerate(actions):
        new_i, new_j = ii + row[0], jj + row[1]
        if 0 <= new_i <= 2 and 0 <= new_j <= 2:
            new_state = []
            for row in state:
                new_state.append(list(row))
            new_state[ii][jj], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[ii][jj]
            #expand_state.append(tuple(new_state))
            expand_state.append(new_state)
            
    ans = [tuple(tuple(row) for row in node) for node in expand_state]
    return ans  
    

In [6]:
"""Check the expand state function"""
from nose.tools import assert_equal, ok_

check_expanded_states(expand_state(((0, 1, 3), (4, 2, 5), (7, 8, 6))),
                     [((4, 1, 3), (0, 2, 5), (7, 8, 6)), ((1, 0, 3), (4, 2, 5), (7, 8, 6))])

check_expanded_states(expand_state(((1, 2, 3), (8, 0, 4), (7, 6, 5))),
                     [((1, 2, 3), (8, 6, 4), (7, 0, 5)),
                     ((1, 0, 3), (8, 2, 4), (7, 6, 5)),
                     ((1, 2, 3), (8, 4, 0), (7, 6, 5)),
                     ((1, 2, 3), (0, 8, 4), (7, 6, 5))])
test_ok()

Test passed!!

## <a name="simple_search"></a>Simple Search

Now you will implement Simple Search, as seen in class, to solve the 8 Puzzle Problem.

We are giving you the class `SearchNode` defined in `search_classes.py`. This class represents a search node in the search tree.

You can create a `SearchNode` by giving it the state it represents and its `SearchNode` parent (or None if it's the root element in the tree). Below is an example of the Serach Node class being used:

In [8]:
# Execute this example code
root_node = SearchNode(((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent_node=None)
children_node = SearchNode(((4, 1, 3), (0, 2, 5), (7, 8, 6)),
                            parent_node=root_node)
print("Root node: %s" % root_node)
print("Children node: %s" % children_node)

Root node: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>
Children node: <SearchNode: state: ((4, 1, 3), (0, 2, 5), (7, 8, 6)), parent: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>>


We also give you the `Path` class, that takes a `SearchNode` and computes the state path from the initial state in the root of the tree to the state of the given `SearchNode`:

In [9]:
# Execute this example code
example_path = Path(children_node)
print("Path of %d states is: %s" % (len(example_path.path), example_path.path))

Path of 2 states is: [((0, 1, 3), (4, 2, 5), (7, 8, 6)), ((4, 1, 3), (0, 2, 5), (7, 8, 6))]


<div class="alert alert-warning">
You will want to look at the `SearchNode` and `Path` definitions in the included **`search_classes.py`** file, as you will need to know what useful properties you can use for the next questions.
</div>

<div class="alert alert-info">
Implement the function `expand_node(self, search_node)` below.
</div>

In [18]:
class PuzzleProblem(object):
    """Class that represents the puzzle search problem."""
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
    def test_goal(self, state):
        return self.goal == state
    def expand_node(self, search_node):
        """Return a list of SearchNodes, having the correct state and parent node."""
        state = search_node.state
        expand_state = []
        # find where the zero is
        for i,row in enumerate(state): # looks at column element within a row
            for j,el in enumerate(row): # looks at row element within the first row
                if el == 0:
                    ii = i
                    jj = j
        #cycle through each action
        #right = [[0,1]] #action, moves the zero in state to the adjacent indice
        #left = [[0,-1]]
        #up = [[1,0]]
        #down = [[-1,0]]
        actions = [[0,1],[0,-1],[1,0],[-1,0]]
        for i,row in enumerate(actions):
            new_i, new_j = ii + row[0], jj + row[1]
            if 0 <= new_i <= 2 and 0 <= new_j <= 2:
                new_state = []
                for row in state:
                    new_state.append(list(row))
                new_state[ii][jj], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[ii][jj]
                #expand_state.append(tuple(new_state))
                expand_state.append(new_state)
                
        ans = [tuple(tuple(row) for row in node) for node in expand_state]
        #for i in enumerate(ans)
        expansion = []
        for i in ans:
            expansion.append(SearchNode(i,parent_node=search_node))        
            
        return expansion
        


In [19]:
"""Check the implementation of PuzzleProblem"""
state_test = ((1, 2, 3), (8, 0, 4), (7, 6, 5))
problem_test = PuzzleProblem(state_test, None)
node_test = SearchNode(state_test, None)
check_expanded_nodes(problem_test.expand_node(node_test),
                     node_test,
                     [((1, 2, 3), (8, 6, 4), (7, 0, 5)),
                     ((1, 0, 3), (8, 2, 4), (7, 6, 5)),
                     ((1, 2, 3), (8, 4, 0), (7, 6, 5)),
                     ((1, 2, 3), (0, 8, 4), (7, 6, 5))])
test_ok()

Test passed!!

### <a name="bfs_implementation"></a>Implement Breadth First Search (45 points)

Finally, you'll implement *Breath First Search*.

Implement the function `breadth_first_search(search_problem)` that takes an instance of the `PuzzleProblem` class that we defined above and returns a tuple of three elements, in the following order:

1. If BFS finds a solution, an instance of the `Path` class containing that solution. If it doesn't, it should return `None` as the first element of the tuple.
2. The number of visited nodes
3. The maximum size of the queue

You should use a **visited list**, as otherwise the number of explored states in this problem will be large.

Also, think that instead of implementing Breadth First Search directly, you could very easily implement a generic *simple_search* function that takes as one of the parameters a function to insert an element in the queue and use this function to implement breath first search, and also depth first search very easily. However, you are not required to do this.

<div class="alert alert-info">
Implement `breadth_first_search(search_problem)` below.
</div>



In [26]:
def breadth_first_search(search_problem):
    start = SearchNode(search_problem.start,parent_node=None) #put PuzzleProblem class into SearchNode class
    Q = [start] # create initial Q list
    V = [start.state] # create visited list
    Length_Q = [len(Q)] # init the queue length to keep track of when it is the biggest
    while Q[0].state != search_problem.goal:
        if len(Q) == 0: #check if queue is empty
            high_q = max(Length_Q)
            print("queue is empty")
            return (None, len(V), high_q)
        else:
            next_nodes = search_problem.expand_node(Q[0]) #expand the node first in the queue
            Q.pop(0) #get rid of the first node in the queue, not equal to goal, already expanded
            #check next_nodes against the visited list, ensure nothing already has been checked
        for index,value in enumerate(next_nodes):
            Q.append(value) #BFS, send children to the back of the line
            #Q.insert(0,i) for DFS,children to the front of the line
            
            
            if next_nodes[index].state in V:
                Q.pop() # may need to change for DFS
            else:
                V.append(value.state) #add state to the visited list
        
        Length_Q.append(len(Q))        
    high_q = max(Length_Q)
    return (Path(Q[0]), len(V), high_q)


### Solve the Puzzle Problem using BFS

Finally, let's use your Breath First Search implementation to solve the 8 Puzzle Problem.
Execute the cell below. If your BFS implementation is correct, you should see the solution printed below.
Don't modify the cell below, as it will be overwritten by our software. If you want to experiment with different states, create new cells below.


In [27]:
# Solve the 8 Puzzle Problem from state:
# +-----+
# |  1 3|
# |4 2 5|
# |7 8 6|
# +-----+
# Don't modify this cell (contents will be overwritten by autograder)
# If you want to experiment with other states, try adding cells below.
# You can try with state: ((1, 8, 2), (0, 4, 3), (7, 6, 5)) for example.
# Remember that not all states have a solution. Try ((8, 1, 2), (0, 4, 3), (7, 6, 5)), for example.
# Be ready to wait, though!
start_state = ((0, 1, 3), (4, 2, 5), (7, 8, 6))
# start_state = ((8, 1, 2), (0, 4, 3), (7, 6, 5))
goal_state = ((1,2,3),(4,5,6),(7,8,0))
problem = PuzzleProblem(start_state, goal_state)

sol, num_visited, max_q = breadth_first_search(problem)
if sol:
    print("Solution found!\n%d states in the solution (%d moves)\n%d states explored.\n%d maximum queue" \
          %(len(sol.path), len(sol.path)-1, num_visited,max_q))
    print("Solution: ")
    for s in sol.path:
        print_state(s)
        print("\n**\n")
else:
    print("No solution after exploring %d states with max q of %d" %(num_visited, max_q))

Solution found!
5 states in the solution (4 moves)
35 states explored.
18 maximum queue
Solution: 
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4   5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5  |
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5 6|
|7 8  |
+-----+

**



### <a name="bfs_vs_dfs"></a> Breadth First Search vs Depth First Search (20 points)

Have you tried using DFS to solve this problem?

Would DFS be a better choice for this problem? What benefits does BFS have over DFS in this problem? Please explain in the cell below (double click on the cell below, remove the text and type your answer). This is a qualitative question, you are not required to give numerical comparisons.

I also created a DFS algorithm. I do not think DFS is a better choice for this problem. The run time for my DFS algorithm ran much longer than the BFS algorithm for the initial state given in the problem. This makes sense given that there are well over 100,000 states in the eight puzzle, and with a branching factor of at most four, the maximum depth to find a solution with DFS could be extremely high. Since the time complexity of DFS is exponential and the worst case is dependent on maximum depth (b^m), DFS will likely be slower to solve the 8-puzzle than BFS, where time complexity is also exponential but with the shallowest depth +1 as the exponent (i.e., b^d+1). Regarding space complexity, although BFS is exponential (b^d+1) and DFS is not exponential (b x m), the memory required to solve it should not be an issue for a modern laptop. Further, although both BFS and DFS are complete and guaranteed to find a solution if there is one (works for DFS since a visited list is used), BFS would find the shortest path (and optimal if we assume each movement of a block is equal in cost) to the solution. Overall, BFS seems to be the better algorithm for the 8-puzzle problem in nearly every respect.  